In [ ]:
import os
from pathlib import Path
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
import logomaker
from pathlib import Path

In [4]:
# Basewise Counts (pos vs neg)
# ----- config -----
CENTER_POS_1IDX = 61
TOTAL_LEN = 121
RADIUS = 15   # ±15 -> 31-nt window
BASES = ["A","C","G","U"]
ROOT = Path("./assay_stats"); ROOT.mkdir(parents=True, exist_ok=True)

ASSAYS = {
    "hek_qb": {
        "pos": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/data/ss_prediction/hek_qb.fasta",
        "neg": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/data/ss_prediction/hek_qb_negatives.fasta",
        "title": "HEK293 PRAISE",
    },
    "hek_trub1_csv": {
        "pos": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/data/ss_prediction/pus7_hek_csv_ENST_U_only.fasta",
        "neg": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/data/ss_prediction/other_pus_hek_csv_enst_downsampled.fasta",
        "title": "HEK293 CMC-based",
    },
    "hela_bidseq": {
        "pos": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/data/ss_prediction/PUS7_positive_sequences_middleU.fa",
        "neg": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/data/ss_prediction/shControl_sequences_middleU.fa",
        "title": "HeLa BID-seq",
    },
    "hela_pseudoseq": {
        "pos": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/data/ss_prediction/pus7_corrected_pse.fasta",
        "neg": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/data/ss_prediction/pus7_negatives_filtered.fasta",
        "title": "HeLa Pseudo-seq",
    },
}


def rna_upper(s: str) -> str:
    return s.strip().upper().replace("T", "U")

def load_windows(fa_path: str, require_center_U: bool):
    """Extract ±R window; enforce length=121; optionally require U at window center."""
    wins, excl = [], Counter()
    c0 = CENTER_POS_1IDX - 1
    for rec in SeqIO.parse(fa_path, "fasta"):
        s = rna_upper(str(rec.seq))
        if len(s) != TOTAL_LEN:
            excl["wrong_length"] += 1; continue
        w = s[c0 - RADIUS : c0 + RADIUS + 1]
        if len(w) != 2*RADIUS + 1:
            excl["window_oob"] += 1; continue
        if require_center_U and w[RADIUS] != "U":
            excl["window_center_not_U"] += 1; continue
        wins.append(w)
    return wins, excl

IDX = {b:i for i,b in enumerate(BASES)}

def counts_matrix(seqs):
    #positions×4 (A,C,G,U)
    L = 2*RADIUS + 1
    M = np.zeros((L, len(BASES)), dtype=int)
    for s in seqs:
        for i, b in enumerate(s):
            j = IDX.get(b)
            if j is not None:
                M[i, j] += 1
    return M

def write_counts_long(Cpos, Cneg, out_csv: Path):
    # counts: position_in_window, base, pos_count, neg_count, pos_total, neg_total
    relpos = np.arange(-RADIUS, RADIUS+1)
    df_pos = pd.DataFrame(Cpos, index=relpos, columns=BASES).reset_index().rename(columns={"index":"position_in_window"})
    df_neg = pd.DataFrame(Cneg, index=relpos, columns=BASES).reset_index().rename(columns={"index":"position_in_window"})
    
    # melt to long
    lp = df_pos.melt(id_vars="position_in_window", var_name="base", value_name="pos_count")
    ln = df_neg.melt(id_vars="position_in_window", var_name="base", value_name="neg_count")
    
    df = lp.merge(ln, on=["position_in_window","base"], how="outer").fillna(0)
    
    # totals per position -> sum across bases
    pos_tot = df_pos[BASES].sum(axis=1).rename("pos_total")
    neg_tot = df_neg[BASES].sum(axis=1).rename("neg_total")

    totals = pd.DataFrame({"position_in_window": relpos, "pos_total": pos_tot, "neg_total": neg_tot})
    df = df.merge(totals, on="position_in_window", how="left")
    
    # ensure types
    for col in ["pos_count","neg_count","pos_total","neg_total"]:
        df[col] = df[col].astype(int)
    df = df.sort_values(["position_in_window","base"]).reset_index(drop=True)
    out_csv.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_csv, index=False)

# ----- main -----
def main():
    for name, cfg in ASSAYS.items():
        outdir = ROOT / name; outdir.mkdir(parents=True, exist_ok=True)

        pos_w, excl_pos = load_windows(cfg["pos"], require_center_U=True)   # positives: U at center
        neg_w, excl_neg = load_windows(cfg["neg"], require_center_U=False)  # negatives: as-is

        if not pos_w or not neg_w:
            print(f"[{name}] skipped (pos={len(pos_w)}, neg={len(neg_w)}) | excl_pos={dict(excl_pos)} | excl_neg={dict(excl_neg)}")
            continue

        Cpos, Cneg = counts_matrix(pos_w), counts_matrix(neg_w)

        # counts-only CSV in the same shape your downstream expects
        out_csv = outdir / f"{name}_basewise_counts.csv"
        write_counts_long(Cpos, Cneg, out_csv)

        # quick check on center bases
        print(f"[{name}] POS center:", Counter(w[RADIUS] for w in pos_w),
              "| NEG center:", Counter(w[RADIUS] for w in neg_w))
        print(" wrote:", out_csv)

if __name__ == "__main__":
    main()
# used in heatmap construction
# Long table of raw counts per position and base for positives and negatives 
# columns: position_in_window, base, pos_count, neg_count, and per-position totals

[hek_qb] POS center: Counter({'U': 104}) | NEG center: Counter({'U': 570})
 wrote: assay_stats/hek_qb/hek_qb_basewise_counts.csv
[hek_trub1_csv] POS center: Counter({'U': 206}) | NEG center: Counter({'U': 412})
 wrote: assay_stats/hek_trub1_csv/hek_trub1_csv_basewise_counts.csv
[hela_bidseq] POS center: Counter({'U': 14}) | NEG center: Counter({'U': 34})
 wrote: assay_stats/hela_bidseq/hela_bidseq_basewise_counts.csv
[hela_pseudoseq] POS center: Counter({'U': 5}) | NEG center: Counter({'U': 23})
 wrote: assay_stats/hela_pseudoseq/hela_pseudoseq_basewise_counts.csv


In [5]:

# Making two-sample heatmaps from *_basewise_counts.csv - counts only

ROOT     = Path("./assay_stats")   
BASES = ["A","C","G","U"]
CMAP     = "bwr"
RADIUS   = 15                    
BBOOT    = 200                  
ALPHA_FLANKS = 1.0     

LABELS = {
    "hek_qb": "HEK293 PRAISE",
    "hek_trub1_csv": "HEK293 CMC-based",
    "hela_bidseq": "HeLa BID-seq",
    "hela_pseudoseq": "HeLa Pseudo-seq",
}

# get counts
def load_counts_long(assay_dir: Path) -> pd.DataFrame:

    cands = sorted(assay_dir.glob("*_basewise_counts.csv"))
    if not cands:
        raise FileNotFoundError(f"No counts CSV in {assay_dir}")
    df = pd.read_csv(cands[0])
    # normalize columns
    if "base" in df.columns:
        df["base"] = df["base"].replace({"T":"U"})
    df["position_in_window"] = pd.to_numeric(df["position_in_window"])
    return df

def split_counts(df_long: pd.DataFrame):
    Ppos = (df_long.pivot(index="position_in_window", columns="base", values="pos_count")
                  .reindex(columns=BASES, fill_value=0).sort_index())
    Pneg = (df_long.pivot(index="position_in_window", columns="base", values="neg_count")
                  .reindex(columns=BASES, fill_value=0).sort_index())
    Tpos = (df_long.drop_duplicates(["position_in_window"])
                   .set_index("position_in_window")["pos_total"].sort_index())
    Tneg = (df_long.drop_duplicates(["position_in_window"])
                   .set_index("position_in_window")["neg_total"].sort_index())
    return Ppos, Pneg, Tpos, Tneg


def mvhypergeom_draw(counts: np.ndarray, n: int) -> np.ndarray:
    counts = counts.astype(int); total = int(counts.sum())
    if n <= 0 or total <= 0: return np.zeros_like(counts)
    if n >= total: return counts.copy()
    try:
        return np.random.default_rng().multivariate_hypergeometric(counts, n)
    except AttributeError:
        out = np.zeros_like(counts); draws = int(n); remaining = total
        for k in range(len(counts)-1):

            if draws <= 0: break
            good = int(counts[k]); bad = int(remaining - good)
            ns = min(draws, remaining)
            take = np.random.hypergeometric(good, bad, ns)
            take = min(take, good, draws)

            out[k] = take; draws -= take; remaining -= good
        out[-1] = min(draws, int(counts[-1])); return out

def dirichlet_probs(C: np.ndarray, alpha: float) -> np.ndarray:
    P = C.astype(float) + alpha

    return P / P.sum(axis=1, keepdims=True)

def equalized_log2_shared_target(Ppos_df, Pneg_df, target: int, alpha_flanks: float = ALPHA_FLANKS):
    pos_idx = Ppos_df.index.to_numpy()
    Cpos, Cneg = Ppos_df.values, Pneg_df.values

    mats = []
    # center row index (position 0)
    center_row = np.where(pos_idx == 0)[0]
    center_row = int(center_row[0]) if center_row.size else None

    for _ in range(BBOOT):
        Spos = np.vstack([mvhypergeom_draw(Cpos[i], target) for i in range(Cpos.shape[0])])
        Sneg = np.vstack([mvhypergeom_draw(Cneg[i], target) for i in range(Cneg.shape[0])])

        
        Ppos = dirichlet_probs(Spos, alpha_flanks) # smoothing on all rows first
        Pneg = dirichlet_probs(Sneg, alpha_flanks)

        if center_row is not None: # then remove smoothing at the center row (avoid fake non-U signal)
            ps = Spos[center_row].sum(); pn = Sneg[center_row].sum()
            if ps > 0: Ppos[center_row, :] = Spos[center_row] / ps
            if pn > 0: Pneg[center_row, :] = Sneg[center_row] / pn

        mats.append(np.log2((Ppos + 1e-12) / (Pneg + 1e-12))) # prevent log(0)

    L2 = np.mean(mats, axis=0) # average across bootstrap replicates
    return pd.DataFrame(L2, index=pos_idx, columns=ALPHABET)

# plot
def shared_vmax_ignore_center(mats):
    vals = []
    
    for df in mats:
        arr = df.drop(index=0, errors="ignore").values
        vals.append(np.nanmax(np.abs(arr)))
    return max(1e-6, float(np.max(vals)))
# map to the y-axis and positions (index) to the x-axis
def plot_panels(mats, titles, outfile: Path):
    vmax = shared_vmax_ignore_center(mats)
    fig, axes = plt.subplots(len(mats), 1, figsize=(8, 2.6*len(mats)), sharex=True, sharey=True)
    axes = np.atleast_1d(axes)
    for ax, Mdf, title in zip(axes, mats, titles):
        im = ax.imshow(Mdf.values.T, aspect="auto", interpolation="nearest",
                       cmap=CMAP, vmin=-vmax, vmax=vmax)
        ax.set_title(title, pad=4)
        ax.set_ylabel("Base"); ax.set_yticks(range(len(BASES))); ax.set_yticklabels(BASES)
        
        pos = Mdf.index.to_numpy(); step = max(1, len(pos)//15)
        ax.set_xticks(range(0, len(pos), step)); ax.set_xticklabels(pos[::step])
    
    axes[-1].set_xlabel("Position (center = 0)")
    
    cax = fig.add_axes([0.92, 0.15, 0.02, 0.7]); cb = fig.colorbar(im, cax=cax); cb.set_label("log2(Ppos / Pneg)")
    fig.tight_layout(rect=[0.04, 0.04, 0.9, 0.96]); fig.savefig(outfile, dpi=300); plt.close(fig)

# find assays
if __name__ == "__main__":

    assays = [d for d in ROOT.iterdir() if d.is_dir() and list(d.glob("*_basewise_counts.csv"))]
    assays = sorted(assays, key=lambda p: list(LABELS).index(p.name) if p.name in LABELS else p.name)

    if not assays:
        raise SystemExit("No *_basewise_counts.csv files found under ./assay_stats")

    titles, PPs, NNs, TPs, TNs = [], {}, {}, {}, {}
    for d in assays:
        df = load_counts_long(d)
        Ppos, Pneg, Tpos, Tneg = split_counts(df)

        Ppos = Ppos.reindex(columns=BASES, fill_value=0)
        Pneg = Pneg.reindex(columns=BASES, fill_value=0)
        PPs[d.name], NNs[d.name], TPs[d.name], TNs[d.name] = Ppos, Pneg, Tpos, Tneg
        titles.append(LABELS.get(d.name, d.name))

    # one shared target for both classes across all assays - the minimum available per-position total
    global_target = min(min(int(TPs[a].min()) for a in PPs), min(int(TNs[a].min()) for a in NNs))
    print("Equalizing both classes to N =", global_target)

    mats = []
    for d in assays: # Bootstrap-equalize, compute mean log2 ratio
        L2 = equalized_log2_shared_target(PPs[d.name], NNs[d.name], target=global_target, alpha_flanks=ALPHA_FLANKS)
        L2.to_csv(d / f"{d.name}_equalN_log2ratio_counts6.csv", index_label="position")
        mats.append(L2)

    out = ROOT / "ALL_equalN_counts_sharedScale6.png"
    plot_panels(mats, titles, out)
    print("Wrote:", out)

Equalizing both classes to N = 5


/local/slurm.14547831/ipykernel_4129884/1981897376.py:114: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0.04, 0.04, 0.9, 0.96]); fig.savefig(outfile, dpi=300); plt.close(fig)


Wrote: assay_stats/ALL_equalN_counts_sharedScale6.png
